### nbdc data

In [1]:
import requests
import pandas as pd
import numpy as np

### get all relevant buoys

In [2]:
text_data = requests.get('https://www.ndbc.noaa.gov/data/stations/station_table.txt').text
rows = text_data.strip().split('\n')

# extracting headers
headers = map(lambda x: x.strip(), rows[0].strip().split('|'))

# remaining data. (skipping with only `#`)
data = [row.strip().split('|') for row in rows[2:]]

df_buoys = pd.DataFrame(data, columns = headers)

### Get buoy data

In [3]:
text_data = requests.get('https://www.ndbc.noaa.gov/data/realtime2/41002.spec').text

# Split the text into rows using newline characters
rows = text_data.strip().split('\n')

# Split each row into columns using spaces
headers = rows[0].strip().split()

data = [row.split() for row in rows[1:]]

# Create a DataFrame from the parsed data
df_buoy_data = pd.DataFrame(data, columns=headers)

# Convert 'Age' column to numeric
# df['Age'] = pd.to_numeric(df['Age'])

# Now you can work with the DataFrame
df_buoy_data

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2023,08,18,06,10,2.2,0.4,19.0,2.2,7.1,SSE,SSW,STEEP,5.6,212
2,2023,08,18,05,40,2.2,0.0,MM,2.2,7.1,MM,SSW,STEEP,5.7,206
3,2023,08,18,05,10,2.1,0.9,17.4,1.9,7.1,SSW,SSW,STEEP,5.6,203
4,2023,08,18,04,40,2.3,0.0,MM,2.3,7.7,MM,S,STEEP,5.9,189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2167,2023,07,04,02,10,0.7,0.6,7.7,0.1,3.4,ENE,SSE,N/A,6.4,77
2168,2023,07,04,01,40,0.7,0.6,7.7,0.1,3.6,E,SSE,N/A,6.4,87
2169,2023,07,04,01,10,0.6,0.6,7.1,0.1,3.7,ENE,S,N/A,6.5,75
2170,2023,07,04,00,40,0.6,0.6,7.7,0.1,4.0,E,SSE,N/A,6.4,88


### Calculate that buoy's angle to Los Angeles

Swell direction is measured with north at 0 degrees, east at 90, etc. 

In [4]:
import math
import re

In [21]:
LA_lat = 33.8847
LA_long = -118.4109

def calculate_angle(coords):
    buoylat, buoylong = parse_coordinates(coords) if isinstance(coords, str) else coords
    LA_lat = 33.8847
    LA_long = -118.4109
    x = buoylong - LA_long
    y = buoylat - LA_lat
    
    if x >= 0:
        return 90 - math.atan(y / x) * 180 / math.pi 
    elif x < 0: 
        return 270 - math.atan(y / x) * 180 / math.pi
    
    #return math.atan(y / x) * 180 / math.pi
    
    
def parse_coordinates(input_string):
    pattern = r'(\d+\.\d+)\s*([NS])\s*(\d+\.\d+)\s*([EW])'
    match = re.search(pattern, input_string)
    if match:
        latitude = float(match.group(1))
        if match.group(2) == 'S':
            latitude = -latitude
        
        longitude = float(match.group(3))
        if match.group(4) == 'W':
            longitude = -longitude
        
        return latitude, longitude
    else:
        return None
    

In [22]:
df_buoys['angletoLA'] = df_buoys.apply(lambda row: calculate_angle(row.LOCATION), axis = 1)

### Valid buoys only 

Buoy